
1. Load AI model

In [ ]:
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np
np.set_printoptions(suppress=True)
model = load_model("./keras_model.h5", compile=False)
class_names = open("./labels.txt", "r").readlines()
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

2. Create API

In [ ]:
from flask import Flask, request, jsonify, send_file
from io import BytesIO
from PIL import Image, ImageOps
import numpy as np

# Initialize Flask app
app = Flask(__name__)

# Assuming the model and class names are already loaded
# `model` is your Keras model, and `class_names` contains the labels

# Define the prediction route
@app.route('/predict', methods=['POST'])
def predict():
    if 'image' not in request.files:
        return jsonify({"error": "No image provided"}), 400
    
    file = request.files['image']

    if file.filename == '' or not file:
        return jsonify({"error": "File name is empty. Please upload a valid image file."}), 400
        
    details = {
        "general": {
            "imageUrl": "general",
            "description": "general",
        },
        "foodwaste": {
            "imageUrl": "foodwaste",
            "description": "foodwaste",
        },
        "hazardous": {
            "imageUrl": "hazardous",
            "description": "hazardous",
        },
        "recycle": {
            "imageUrl": "recycle",
            "description": "recycle",
        },
    }

    img = Image.open(file)
    img = img.convert("RGB")
    
    # Resize and preprocess the image as required by the model
    img = ImageOps.fit(img, (224, 224), Image.Resampling.LANCZOS)
    img_array = np.asarray(img)
    normalized_image_array = (img_array.astype(np.float32) / 127.5) - 1  # Normalization
    
    # Model expects an input of shape (1, 224, 224, 3)
    data = np.expand_dims(normalized_image_array, axis=0)
    
    # Run the model prediction
    prediction = model.predict(data)
    
    # Get the predicted class name
    predicted_class = class_names[np.argmax(prediction[0])].strip()
    
    # Convert the image to a byte stream for sending back
    img_io = BytesIO()
    img.save(img_io, 'PNG')
    img_io.seek(0)
    
    # Send back the prediction and image
    return jsonify({
        "type": predictedType,
        "imageUrl": details[predictedType]["imageUrl"],  # Correct access method
        "description": details[predictedType]["description"],  # Correct access method
    }), 200

# Start the Flask server
if __name__ == '__main__':
    app.run(port=3000)